In [1]:
import json
import openai

Creating Flask App

In [2]:
from flask import Flask, redirect, url_for, request, Response
from flask_mail import Mail, Message 

app = Flask(__name__)

@app.route("/")
def home():
    return 'Kathan Here!'

Reading Email Credentials

In [3]:
f = open("email_cred.json", "r")

cred = json.load(f)

email_address = cred.get("email_address")
email_password = cred.get("email_password")

print(cred);

{'email_address': 'kathan.naik@aethereus.com', 'email_password': 'Bits@20074985'}


Setting Email Config

In [4]:
app.config['MAIL_SERVER']='smtp.office365.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USERNAME'] = email_address
app.config['MAIL_PASSWORD'] = email_password
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USE_SSL'] = False
mail = Mail(app)

Reading Pulze Keys

In [5]:
f = open("key.txt", "r")

openai.api_key = f.readline()
openai.api_base = "https://api.pulze.ai/v1"

f.close()

Defining System Instruction

In [6]:
instructiones = '''User will give an email text as message content. Your task is to retrive email signature from that email. You should follow below steps 
1) figure out the name of person sending the mail
2) find the name of the organization or company the person belongs to if mentioned any
3) check if there is an mobile number the person writing email might have given
4) return the response message content in JSON like format as below
    {
        "name": "name that you have figured out"
        "org": "organization/company"
        "number": mobile number found
    }
5) you can keep fields empty by putting 'Null' if you do not find any of them'''

Function to Call Pulze API

In [7]:
def pulze_llm(mail):
    chat_response = openai.ChatCompletion.create(
        model="pulze-v0",
        max_tokens=100,
        messages=[
            {"role": "system", "content": instructiones},
            {"role": "user", "content": mail}
        ],
    )

    print(chat_response)
    return chat_response.choices[0].message.content

POST Request Handler

In [8]:
@app.route("/mailpost", methods=["POST"])
def login():
    if request.method == "POST":
        user_address = request.form["address"]
        user_email = request.form["email"]
        access_password = request.form["password"]
        
        if(access_password == "Aethereus@123"):
            msg = Message( 
                'Email-Conformation', 
                sender = email_address, 
                recipients = [user_address] 
               ) 
            msg.body = 'A case has been raised based on your email and our team will be contacting you soon. Thank you for reaching out!'
            mail.send(msg) 
            
            r = Response(response=pulze_llm(user_email), status=200, mimetype="application/json")
            r.headers["Content-Type"] = "application/json; charset=utf-8"
            return r
        else:
            return "invalid request"

Running the App

In [ ]:
if __name__ == "__main__":
    app.run(host="localhost", port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8080
Press CTRL+C to quit
127.0.0.1 - - [07/Nov/2023 11:14:06] "POST /mailpost HTTP/1.1" 200 -


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n  \"name\": \"Dr. Chaitali Bhattacharya\",\n  \"org\": \"INDO-US SCIENCE & TECHNOLOGY FORUM\",\n  \"number\": \"91-11-42691718\"\n}",
        "role": "assistant"
      }
    }
  ],
  "created": 1699335701332,
  "id": "4472f974-6e38-4230-bac1-663332a9cb0c",
  "metadata": {
    "app_id": "f05910bc-7f61-4623-b323-4f7981526ecc",
    "category": "Education & Learning",
    "cost_savings": {
      "completion_tokens": 0.00261,
      "prompt_tokens": 0.025172,
      "total_tokens": 0.027782
    },
    "costs": {
      "completion_tokens": 9e-05,
      "prompt_tokens": 0.000868,
      "total_tokens": 0.000958
    },
    "latency": 1.208,
    "max_tokens": 100,
    "model": {
      "model": "gpt-3.5-turbo",
      "namespace": "openai/gpt-3.5-turbo",
      "provider": "openai"
    },
    "retries": 0,
    "score": 0.333,
    "scores": {
      "best_models": [
        {
          "ope